# Evaluating Semantic Relevance Between Queries and LOINC Codes using Text Similarity Analysis

In [279]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import os


Possible queries:
- glucose in blood
- bilirubin in plasma
- White blood cells count

In [280]:
#Dataset file
###################### modify each time you want to perform a new query on a different sheet of the file excel
query = 'White blood cells count'  

Dataset_path = r'/home/gabriele/Desktop/Polima/Biomedical Informatic/MachineLearningRanking-main/data/loinc_dataset-v2.xlsx'
Dataset = pd.read_excel(Dataset_path, header=2, sheet_name=query) #removing the first 2 columns (metadata)

print(Dataset.head())
print(Dataset.columns)

Dataset_loinc_num = Dataset.pop('loinc_num')

print(Dataset.columns)

  loinc_num                                   long_common_name  \
0   33870-7  Bilirubin.total [Presence] in Unspecified spec...   
1   29265-6  Calcium [Moles/volume] corrected for albumin i...   
2   14423-8    Bilirubin.total [Mass/volume] in Synovial fluid   
3   23658-8                  Other Antibiotic [Susceptibility]   
4   19000-9                        Vancomycin [Susceptibility]   

                        component    system property  
0                       Bilirubin       XXX    PrThr  
1  Calcium^^corrected for albumin  Ser/Plas     SCnc  
2                       Bilirubin  Synv fld     MCnc  
3                  Antibiotic XXX   Isolate     Susc  
4                      Vancomycin   Isolate     Susc  
Index(['loinc_num', 'long_common_name', 'component', 'system', 'property'], dtype='object')
Index(['long_common_name', 'component', 'system', 'property'], dtype='object')


The interest is on understand how much is initially relevant a document with respect to a query. In order to do this we can use all the informations available from the dataset (common_long_name, component, system, property) to build an initial ranking using a model.

The model that can be used is a pretrained Word2Vec model, tuned specifically for this task. The Word2Vec model is useful because it can understand the patterns of semantic relationships and contextual similarity between biomedical terms, even when faced with abbreviations, synonyms, or paraphrasing.

Before doing this we need to manage the dataset in order to have clear sentences, joining the columns of long_common_name, component, system, property

In [281]:
Dataset = Dataset.to_numpy()

for i in range(3):
    print(Dataset[i])

print()
print()

corpus = [" ".join(row) for row in Dataset]
print(corpus)

title_to_loinc = dict(zip(corpus, Dataset_loinc_num))
print(title_to_loinc)



#Cleaning of phrases
sentences = []
for sentence in corpus:
    sentence = sentence.lower()
    sentence = sentence.replace('/',' ')
    cleaned_sentence = re.sub(r'[^a-z\s]', r' ', sentence)
    tokens = [token for token in cleaned_sentence.split() if token]
    sentences.append(tokens)

print(sentences)

['Bilirubin.total [Presence] in Unspecified specimen' 'Bilirubin' 'XXX'
 'PrThr']
['Calcium [Moles/volume] corrected for albumin in Serum or Plasma'
 'Calcium^^corrected for albumin' 'Ser/Plas' 'SCnc']
['Bilirubin.total [Mass/volume] in Synovial fluid' 'Bilirubin' 'Synv fld'
 'MCnc']


['Bilirubin.total [Presence] in Unspecified specimen Bilirubin XXX PrThr', 'Calcium [Moles/volume] corrected for albumin in Serum or Plasma Calcium^^corrected for albumin Ser/Plas SCnc', 'Bilirubin.total [Mass/volume] in Synovial fluid Bilirubin Synv fld MCnc', 'Other Antibiotic [Susceptibility] Antibiotic XXX Isolate Susc', 'Vancomycin [Susceptibility] Vancomycin Isolate Susc', 'Glucose [Moles/volume] in Serum or Plasma Glucose Ser/Plas SCnc', 'Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma Aspartate aminotransferase Ser/Plas CCnc', 'Cefazolin [Susceptibility] Cefazolin Isolate Susc', 'Hepatitis B virus DNA [#/volume] (viral load) in Serum by Probe with signal amplification He

In [282]:
#Libraries for Word2Vec
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences, vector_size=50, window=2, min_count=2, workers=4, epochs=10)
print(w2v_model.wv.index_to_key)



['in', 'volume', 'ser', 'plas', 'serum', 'or', 'plasma', 'blood', 'mass', 'mcnc', 'moles', 'bld', 'glucose', 'bilirubin', 'scnc', 'type', 'isolate', 'susceptibility', 'susc', 'calcium', 'group', 'albumin', 'total', 'product', 'aminotransferase', 'abo', 'protein', 'c', 'indirect', 'choriogonadotropin', 'enzymatic', 'by', 'ncnc', 'reactive', 'acnc', 'cholesterol', 'leukocytes', 'activity', 'rh', 'chloride', 'ionized', 'ccnc', 'lymphocytes', 'ampicillin', 'presence', 'carbon', 'dioxide', 'glucuronidated', 'xxx', 'specific', 'bpu', 'b', 'unit', 'post', 'phosphatase', 'antibiotic', 'aspartate', 'crossmatch', 'unspecified', 'vancomycin', 'specimen', 'globulin', 'levofloxacin', 'for', 'antiglobulin', 'major', 'corrected', 'monocytes', 'fluid', 'test', 'complement', 'reagent', 'hematocrit', 'bilirubinate', 'tyrosine', 'body', 'temperature', 'fld', 'staphylococcus', 'cefazolin', 'hepatitis', 'carcinoembryonic', 'cortisol', 'po', 'g', 'screen', 'antibody', 'sulfamethoxazole', 'trimethoprim', 'ni

Now the word2vec model is trained, so there is the possibility to represent the words as vectors of numer (in this case of size 100).

To get a representation of a title is just needed to calculate the mean vector of all the sentences.

In [283]:
from nltk.tokenize import word_tokenize
# Function to get the vector representation of a phrase by averaging word vectors
def sentence_vector(phrase, model):
    words = word_tokenize(phrase.lower())  # Tokenize the phrase into words
    #print(words)
    word_vectors = [model.wv[word] for word in words if word in model.wv]  # Get vectors for words in the model's vocabulary
    #print(len(word_vectors))
    #print(word_vectors)

    #in this moment we have 300 numbers for each word, then 6*300 numbers (6 matrices)
    # due to the fact that we have 6 words

    if word_vectors:
        #print(len((np.mean(word_vectors, axis=0))))
        #we are summing 6 matrices of dimension 1x300, and then dividing everything for 6

        return np.mean(word_vectors, axis=0)  # Average the word vectors
    else:
        return np.zeros(model.vector_size)  # Return a zero vector if no words are in the model



Now it is possible to apply this function easily to all the sentences and get a measure of similarity (like using cosine similarity)

In [284]:
from sklearn.metrics.pairwise import cosine_similarity


query_vec = sentence_vector(query, w2v_model)

print(query)
#print(query_vec)

sentences_vecs = [sentence_vector(" ".join(sentence), w2v_model) for sentence in sentences]
#print(sentences_vecs)

similarities = cosine_similarity([query_vec], sentences_vecs).flatten()

# Ranking
ranking = np.argsort(similarities)[::-1]



results = []
for i in ranking:
    title = corpus[i]
    loinc_num = title_to_loinc.get(title, "NA")  # prende il numero LOINC, "NA" se non trovato
    relevance = similarities[i]
    results.append([title, loinc_num, relevance])
    #print(f"{title} → LOINC: {loinc_num} → relevance: {relevance:.3f}")

ranking_df = pd.DataFrame(results, columns=['long_common_name', 'loinc_num', 'similarity'])

print(ranking_df.head())

folder_path = os.path.join("data")
os.makedirs(folder_path, exist_ok=True)  # Crea la cartella se non esiste

# Definizione del path completo del file
output_path = os.path.join(folder_path, f"ranking {query}.xlsx")

ranking_df.to_excel(output_path, index=False)



White blood cells count
                                    long_common_name loinc_num  similarity
0  ABO group [Type] in Blood from Blood product u...   14578-9    0.664988
1  Blood group antibody screen [Presence] in Seru...     890-4    0.663357
2  Blood product unit ID [#] Blood product unit I...     934-0    0.588803
3    Blood product type Blood product type ^BPU Type     933-2    0.569553
4  Blood product disposition [Type] Blood product...     925-8    0.558776
